# Module: `upload_physio`
 
> The purpose of this module

## 🏗 Infrastructure
> Module dependencies 

In [9]:
import io
import param
import panel as pn
import numpy as np
import pandas as pd
import holoviews as hv
import random
import plotly.express as px

pn.extension('plotly', 'tabulator', sizing_mode="stretch_width")

## 🛠 Tool: Panel

In [2]:
from panel.template import DefaultTheme

DefaultTheme.find_theme(pn.template.MaterialTemplate)

panel.template.material.MaterialDefaultTheme

In [3]:
from panel.template.theme import Theme
from bokeh.themes import DARK_MINIMAL

class DarkTheme(Theme):
    """
    The DarkTheme provides a dark color palette
    """

    bokeh_theme = param.ClassSelector(class_=(Theme, str), default=DARK_MINIMAL)

class MaterialDarkTheme(DarkTheme):

    # css = param.Filename() Here we could declare some custom CSS to apply
    
    # This tells Panel to use this implementation
    _template = pn.template.MaterialTemplate

In [15]:
class SignalApp(param.Parameterized):
    data = param.DataFrame()
    file_input = param.Parameter()
    
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.plotly_pane = pn.pane.Plotly(height=400, sizing_mode="stretch_width")
        
    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        if value:
            string_io = io.StringIO(value.decode("utf8"))
            # self.data = pd.read_csv(string_io, parse_dates=["t"])
            self.data = pd.read_csv(string_io)
        else:
            print("error")
            
    @pn.depends('data', watch=True)
    def get_plot(self):
        df = self.data
        if df is None:
            return
        # assert ("ECG" in df.columns), "no columns ecg"
        # df = (df.loc[df['ECG'] != 'Invalid/Calib']).copy(deep=True)
        df['ECG'] = df['ECG'].astype(float)
        p = px.line(df, x = pd.Series(range(0, len(df))), y = 'ECG', title='ECG(t)')
        self.plotly_pane.object = p
        
    def view(self):
        return pn.Column(
            "## Upload and Plot Data",
            self.file_input,
            self.plotly_pane,
        )

signal_app = SignalApp()
signal_app_view = signal_app.view()
# signal_app_view

In [32]:
xs = np.linspace(0, np.pi)
freq = pn.widgets.FloatSlider(name="Frequency", start=0, end=10, value=2)
phase = pn.widgets.FloatSlider(name="Phase", start=0, end=np.pi)

@pn.depends(freq=freq, phase=phase)
def sine(freq, phase):
    return hv.Curve((xs, np.sin(xs*freq+phase))).opts(
        responsive=True, min_height=400)

@pn.depends(freq=freq, phase=phase)
def cosine(freq, phase):
    return hv.Curve((xs, np.cos(xs*freq+phase))).opts(
        responsive=True, min_height=400)

In [31]:
from panel.template import DarkTheme

dark_material = pn.template.MaterialTemplate(title='🎵Body', theme=DarkTheme)

dark_material.sidebar.append(freq)
dark_material.sidebar.append(phase)

dark_material.main.append(
    pn.Column(
        pn.Row(
            pn.Card(hv.DynamicMap(sine), title='Sine'),
            pn.Card(hv.DynamicMap(cosine), title='Cosine'),
        ),
        # pn.Row(
        #     pn.Card(hv.DynamicMap(sine), title='Sine'),
        #     pn.Card(hv.DynamicMap(cosine), title='Cosine'),
        # ),
        pn.Card(signal_app_view),
        sizing_mode='stretch_both'
    )

)
dark_material.show()

Launching server at http://localhost:55940
